In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
Load the book data and create a sparse matrix of user ratings
book_sparse = book_pivot

# Compute the cosine similarity matrix between the books
book_sim = cosine_similarity(book_sparse)

# Train a nearest neighbors model using the similarity matrix
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(book_sim)

In [ ]:
# Load the book metadata and vectorize it using TF-IDF
book_data = books
book_data.fillna('', inplace=True)  # Replace NaNs with empty strings
vectorizer = TfidfVectorizer()
book_features = vectorizer.fit_transform(book_data['title'] + " " + book_data['author'])

In [ ]:
# Compute the cosine similarity matrix between the book features
feature_sim = cosine_similarity(book_features)

In [ ]:
Define a function that combines the similarity scores from both models
def hybrid_similarity(user_index, book_index):
    # Compute the collaborative filtering similarity score
    cf_sim = knn.kneighbors([book_sim[book_index]], return_distance=False)[0]
    cf_sim = np.mean([book_sim[book_index][i] for i in cf_sim])

    # Compute the content-based filtering similarity score
    cb_sim = feature_sim[book_index][user_index]

    # Combine the similarity scores using a weighted average
    similarity = 0.8 * cf_sim + 0.2 * cb_sim

    return similarity

In [ ]:
# Define a function that recommends books to a user
def recommend_books(user_index, n_recommendations=10):
    # Compute the similarity scores between the user and all the books
    user_sim = np.apply_along_axis(hybrid_similarity, axis=1, arr=book_sparse, book_index=user_index)

    # Sort the books by their similarity scores
    sorted_books = np.argsort(user_sim)[::-1]

    # Get the top recommended books
    recommendations = sorted_books[:n_recommendations]

    return recommendations